In [1]:
%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pylab

## 1. Reading in a Fixed Width Formatted File and Converting it to a periodic Time Series.

In [3]:
#lets read in a fixed width file
data = pd.read_fwf('data/ao_monthly.txt', header = None)
data.head()

,0,1,2
0,1950,1,-0.060310
1,1950,2,0.626810
2,1950,3,-0.008128
3,1950,4,0.555100
4,1950,5,0.071577


In [19]:
#lets format this data better when calling pd.read_fwf
"""
We specify the columns containing time data(here year from col 0, and month from col 1)
We also tell pandas to infer the date format from the cols specified in parse_dates
"""
col_names = [2]
CleanerData = pd.read_fwf('data/ao_monthly.txt', header = None, use_cols = col_names, index_col = 0, parse_dates=[[0,1]], infer_datetime_format=True)
#in the following cell we saw that it infered the us format of mm/dd

In [21]:
#rename the columns to for better comprehension
CleanerData.columns = ['Measurement']
CleanerData.index.names=['Date']
CleanerData.head()

,Measurement
Date,
1950-01-01,-0.060310
1950-02-01,0.626810
1950-03-01,-0.008128
1950-04-01,0.555100
1950-05-01,0.071577


In [22]:
#Now that the data format is easier to work with, we can now extract some useful information

#Lets first find the timespan that the data deals with
print(min(CleanerData.index))
print(max(CleanerData.index))

1950-01-01 00:00:00
2016-06-01 00:00:00


In [23]:
#we see we've got timestamps as output, we dont need timestamps, since this seems to be data for a period(here monthly)
CleanerData.to_period().head()

#we can see that the data type is a period, since pandass does not bother setting a date for each of the dates.

,Measurement
Date,
1950-01,-0.060310
1950-02,0.626810
1950-03,-0.008128
1950-04,0.555100
1950-05,0.071577


In [26]:
CleanerData = CleanerData.to_period()
CleanerData.head()

,Measurement
Date,
1950-01,-0.060310
1950-02,0.626810
1950-03,-0.008128
1950-04,0.555100
1950-05,0.071577


## 2. Comparing different ways to parse dates, when reading in files using pandas read function

    Having to parse dates from a small file probably does not require attention 
    to the way you parse dates. But when it comes to big data, it is better to 
    be thoughtful of this. Sometimes inference could be faster, sometimes passing
    in a format string, and so on.

In [27]:
import timeit
#strptime converts the string representation to a datetime object 
#It takes in two values (col 0(year) and col 1(month) and pass the formatted string to strptime)
dateparse = lambda x, y: pd.datetime.strptime('%s-%s'%(x,y), '%Y-%m')

#let's time different ways of reading in fwf data
%timeit data = pd.read_fwf('data/ao_monthly.txt', header=None, index_col = 0, parse_dates=[[0,1]], date_parser=dateparse)

12.8 ms ± 95.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
#lesson - you being helpful, by providing a custom solution, does not always work
%timeit data = pd.read_fwf('data/ao_monthly.txt', header=None, index_col = 0, parse_dates=[[0,1]])

4.81 ms ± 42.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
#lesson - try a few things out with the data, find out which one is the fastest
#because the time difference for reading in data will be relevant for larger datasets.
%timeit data = pd.read_fwf('data/ao_monthly.txt', header=None, index_col = 0, parse_dates=[[0,1]], infer_datetime_format=True)

5.08 ms ± 39 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
"""
Suppose you've already read in the data(maybe it was pickled or something. 
You now want to make a timestamp out of multiple columns in the dataframe.
Enter datetime.
"""
sampleData = pd.DataFrame({'year':[2015, 2016], 'month':[2,3], 'day':[25, 26], 'hour':[12, 13]})
sampleData

,year,month,day,hour
0,2015,2,25,12
1,2016,3,26,13


In [32]:
#the to_datetime function automatically inferred what each time element each column contained
#we didn't need to write explicit code for it.
"""
however, if the dataframe had a column which did not have time related column name, 
the above would throw an error.
"""
pd.to_datetime(sampleData)

0   2015-02-25 12:00:00
1   2016-03-26 13:00:00
dtype: datetime64[ns]

## 3. Truncating

In [33]:
#truncating values
series = pd.Series(range(10), index = pd.date_range('4/22/2019', freq='M', periods=10))
series.head()

2019-04-30    0
2019-05-31    1
2019-06-30    2
2019-07-31    3
2019-08-31    4
Freq: M, dtype: int64

In [34]:
#I think truncating is another way of removing the dates you dont want.
series.truncate(before='4/22/2019', after='7/15/2019')

#truncating preserves frequency

2019-04-30    0
2019-05-31    1
2019-06-30    2
Freq: M, dtype: int64

In [36]:
#you can truncate in ways that does not preserve frequency (freq=None in result),
#bacause there is no obvious way of creating a frequency out of these timestamps
series[[1,6,7]].index

DatetimeIndex(['2019-05-31', '2019-10-31', '2019-11-30'], dtype='datetime64[ns]', freq=None)

In [37]:
#But pandas will try to preserve frequency whenever possible.(see freq = 2M)
series[0:10:2]

2019-04-30    0
2019-06-30    2
2019-08-31    4
2019-10-31    6
2019-12-31    8
Freq: 2M, dtype: int64